<a href="https://www.kaggle.com/code/alemazz11/used-cars-analysis-and-price-prediction?scriptVersionId=294743877" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Data Cleaning


In [ ]:
df = pd.read_csv("/kaggle/input/cars-europe/fullGas.csv")
df.head(10)

In [ ]:
df.info()

Checking for duplicates

In [ ]:
df.duplicated().sum() 

## 1.1 Handling missing values

Let's start by removing the columns
* Fuel_Consumption_l  : Almost 70% of data missing
* Gears               : Not so important for analysis and almost half of data missing
* Previous_Owners     : Almost 50% of data missing

In [ ]:
df.drop(columns = ["Fuel_Consumption_l", 'Gears', "Previous_Owners"], inplace=True)


In [ ]:
df.info()

In [ ]:
df[df['Model'].isna()]

Since we want to look at Model in our Data Exploration and there are many null values in other columns, i'll remove those 200 rows.

In [ ]:
df.dropna(subset=['Model'], inplace=True)

In [ ]:
df[df['Year'].isna()]

We see year is missing where the condition is New, so let's change year to 2025(when the dataset was scraped) where the Condition is new, and then remove the remaining rows

While we are at it, let's change columns type to int64

In [ ]:
df['Year'] = df['Year'].astype('Int64')

filter_year = (df['Year'].isna()) & (df['Condition'] == 'New')
df.loc[filter_year, 'Year'] = 2025

print(f"Updated {filter_year.sum()} rows")
print(f"Removed {df['Year'].isna().sum() } rows ")
df.dropna(subset=['Year'], inplace=True)

Let's look at country

In [ ]:
df[df['Country'].isna()]

To keep those rows, I'll set Country to unknown in the rows it's missing

In [ ]:
df['Country'] = df['Country'].fillna('Unknown')

Let's look at drivetrain now

In [ ]:
df[df['Drivetrain'].isna()]

Let's update the rows where we find a match between make and model having the info about drivetrain,
and set to unknown the remaining rows

In [ ]:
# grouping by make, model getting the mode of drivetrain by pair and then applying it to na values
# to corresponding pairs
drivetrain_by_make_model = df.groupby(['Make', 'Model'])['Drivetrain'].transform(lambda x: x.mode()[0] 
                                                                   if not x.mode().empty else None)
df['Drivetrain'] = df['Drivetrain'].fillna(drivetrain_by_make_model)

print(f"{df['Drivetrain'].isna().sum()} rows left with unknown drivetrain")

df['Drivetrain'] = df['Drivetrain'].fillna("Unknown")

Now let's apply the same approach to the remaining columns and check where we are at.

In [ ]:
engine_by_make_model = df.groupby(['Make', 'Model'])['Engine_Size_cc'].transform('median')
df['Engine_Size_cc'] = df['Engine_Size_cc'].fillna(engine_by_make_model)

for col in ['Cylinders', 'Seats', 'Doors']:
    col_by_make_model = df.groupby(['Make', 'Model'])[col].transform(
        lambda x: x.mode()[0] 
        if not x.mode().empty else None
    )
    df[col] = df[col].fillna(col_by_make_model)

df['Color'] = df['Color'].fillna('Unknown')
df['Upholstery'] = df['Upholstery'].fillna('Unknown')

df.info()



Let's remove the few hundred rows that still have missing values, change the type of columns
where it's not what it's supposed to be, and we should be good.

In [ ]:
df.dropna(inplace=True)

df['Year'] = df['Year'].astype('int64')
df['Engine_Size_cc'] = df['Engine_Size_cc'].astype('int64')
df['Cylinders'] = df['Cylinders'].astype('int64')
df['Seats'] = df['Seats'].astype('int64')
df['Doors'] = df['Doors'].astype('int64')

df.info()

Perfect, we only removed about 5% of rows but still kept our dataframe with meaningful data.
Lastly let's fix the outliers by looking at min and max values we got.

In [ ]:
df.describe()

To remove the outliers, we need to know that the low values for enginesize and cylinders are due to the fact we have electric cars in the dataset.

In [ ]:
df = df[(df['Price'] >= 500) & (df['Price'] < 9000000)]
df = df[df['Mileage_km'] < 1000000]
df = df[df['Engine_Size_cc'] < 8000]
df = df[df['Power_hp'] > 10]
df = df[(df['Seats'] > 1 )& (df['Seats'] < 10)]
df = df[(df['Doors'] > 1 )& (df['Doors'] < 7)]

print(f"Data cleaned! Remaining rows: {len(df)}")
df.describe()

In [ ]:
df.to_csv("df_clean.csv", index=False)

# 2. Data Exploration

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("ticks")
sns.set_palette("muted")

## 2.1 Price Analysis

Let's start by looking at price distribution

In [ ]:
sns.displot(df, x="Price", kde=True, log_scale=True)

We see the data is right skewed, with most of cars ranging from 10k to 30k, which is pretty normal for the car market.

In [ ]:
plt.figure(figsize=(12, 8))
sns.boxplot(data=df, x="Body", y ="Price", log_scale=True,hue ="Body")

Let's give a look to which are the most expensive cars and which are the cheapest

In [ ]:
df['Make_Model'] = df['Make'] + " " + df['Model']

top_10_expensive_cars = df.nlargest(10, 'Price')
bottom_10_cheap_cars = df.nsmallest(10, 'Price')

fig, axes = plt.subplots(2, 1, figsize=(10, 12))


axes[0].barh(range(10), top_10_expensive_cars['Price'], color='salmon')
axes[0].set_yticks(range(10))
axes[0].set_yticklabels(top_10_expensive_cars['Make_Model'])
axes[0].invert_yaxis() 
axes[0].set_title("Top 10 Most Expensive Listings")
axes[0].set_xlabel("Price (M€)", fontsize=12)
axes[0].set_ylabel("Car Model", fontsize=12)

axes[1].barh(range(10), bottom_10_cheap_cars['Price'], color='seagreen')
axes[1].set_yticks(range(10))
axes[1].set_yticklabels(bottom_10_cheap_cars['Make_Model'])
axes[1].invert_yaxis() 
axes[1].set_title("Top 10 Cheapest Listings")
axes[1].set_xlabel("Price (€)", fontsize=12)
axes[1].set_ylabel("Car Model", fontsize=12)

plt.show()

We can see how ferrari dominates the charts for the expensive cars, while in the cheap we find Lancia,Chevrolet and Honda.


## 2.2 Depreciation Factor (Age and Mileage) Analysis

First, let's create categories of budget, mid-range, premium and luxury cars, to look how price is affected in each category.

* Budget : Price < 12k€
* Mid-Tier : 12k€ <= Price < 25k€ 
* Premium : 25k€ <= Price <  60k€ 
* Luxury : Price >= 60k€


In [ ]:
def get_category(price):
    if price < 12000:
        return 'Budget'
    elif price < 25000:
        return 'Mid-Range'
    elif price < 55000:
        return 'Premium'
    else:
        return 'Luxury'

df['Category'] = df['Price'].apply(get_category)

print(df['Category'].value_counts().sort_index())

In [ ]:

temp_df = df.copy()
temp_df['Mileage_Bracket'] = pd.cut(temp_df['Mileage_km'], 
                                     bins=[0, 25000, 50000, 75000, 100000, 150000, 200000, 300000],
                                     labels=['0-25k', '25-50k', '50-75k', '75-100k', '100-150k', '150-200k', '200k+'])

# Calculate average price 
price_by_mileage = temp_df.groupby(['Category', 'Mileage_Bracket'], observed=True)['Price'].mean().reset_index()


plt.figure(figsize=(14, 8))
for category in ['Budget', 'Mid-Range', 'Premium', 'Luxury']:
    data = price_by_mileage[price_by_mileage['Category'] == category].copy()
    baseline_price = data['Price'].iloc[0]  # First mileage bracket (0-25k)
    data['Pct_Loss'] = ((baseline_price - data['Price']) / baseline_price) * 100
    
    plt.plot(data['Mileage_Bracket'], data['Pct_Loss'], 
             marker='o', linewidth=2.5, markersize=8, label=category)

plt.xlabel('Mileage Bracket', fontsize=12)
plt.ylabel('Price Loss (%)', fontsize=12)
plt.title('Percentage Price Loss by Mileage Across Categories', fontsize=14, fontweight='bold')
plt.legend(title='Price Category', fontsize=11)
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='black', linestyle='--', linewidth=1)
plt.tight_layout()
plt.show()


Now to look at age let's create another column of car age doing 2026-year

In [ ]:
df['Age'] = 2026 - df['Year']

In [ ]:
temp_df = df.copy()
temp_df['Age_Brackets'] = pd.cut(temp_df['Age'], 
                                 bins=[0, 1, 2, 3, 4, 5, 6, 7, 8, 100],  # Fixed: proper bin edges
                                 labels=['1', '2', '3', '4', '5', '6', '7', '8', '9+'])

# Calculate average price 
price_by_age = temp_df.groupby(['Category', 'Age_Brackets'], observed=True)['Price'].mean().reset_index()

plt.figure(figsize=(14, 8))
for category in ['Budget', 'Mid-Range', 'Premium', 'Luxury']:
    data = price_by_age[price_by_age['Category'] == category].copy()
    baseline_price = data['Price'].iloc[0]  # First age bracket
    data['Pct_Loss'] = ((baseline_price - data['Price']) / baseline_price) * 100
    
    plt.plot(data['Age_Brackets'], data['Pct_Loss'],  
             marker='o', linewidth=2.5, markersize=8, label=category)

plt.xlabel('Car Age (years)', fontsize=12)
plt.ylabel('Price Loss (%)', fontsize=12)
plt.title('Percentage Price Loss by Car Age Across Categories', fontsize=14, fontweight='bold')  # Fixed: changed title
plt.legend(title='Price Category', fontsize=11)
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='black', linestyle='--', linewidth=1)
plt.tight_layout()
plt.show()

In those two graphs we can see how mileage has a big impact on all categories, while age affects the middle and luxury range the most.

Finally let's look and see if power correlates linearly to price.

In [ ]:
sns.jointplot(data=df, x='Power_hp', y='Price', kind='reg',line_kws={'color':'red'}, height=10)
plt.xlabel("Power")
plt.ylabel("Price (€)")
plt.show()

From this graph we can see that power correlates well with cars under 100k, but we got some outliers of cars with high price value but not so much in power.

## 2.3 Features Analysis 

Let's now look at some pie plots of categorical values and how they are distributed

In [ ]:
columns_to_plot = ['Full_Service_History', 'Fuel_Type', 'Body', 'Non_Smoker_Vehicle', 
                   'Gearbox', 'Drivetrain']

fig, axes = plt.subplots(2, 3, figsize=(20, 10))
axes = axes.flatten()

for i, col in enumerate(columns_to_plot):
    data = df[col].value_counts()
    percentages = (data / data.sum()) * 100
    data_filtered = data[percentages > 2]
    
    other_count = data[percentages <= 2].sum()
    other_percentage = (other_count / data.sum()) * 100
    if other_count > 0 and other_percentage >= 0.5:
        data_filtered['Other'] = other_count
    
    # Pie chart WITHOUT labels, only percentages
    wedges, texts, autotexts = axes[i].pie(data_filtered.values, 
                                            autopct='%1.1f%%', startangle=90, 
                                            colors=sns.color_palette('Set2', len(data_filtered)))
    
    # Add legend with BIGGER font
    axes[i].legend(data_filtered.index, loc='center left', bbox_to_anchor=(1, 0, 0.5, 1), 
                   fontsize=12)  # Changed from 8 to 10
    axes[i].set_title(f'{col} Distribution', fontsize=15, fontweight='bold')

plt.tight_layout()
plt.show()

From this graph we can see that there are more automatic cars than manual, front wheel drive and gasoline and, strangely, the most common type is off-road (SUV's).

# 3. Price Prediction

In [ ]:
import scipy.stats

df = df.drop(['Image_url', 'Seller','Year', 'Make_Model', 'Category'], axis=1)

## 3.1 Feature Engineering 

Let's add some columns.

In [ ]:
df['Mileage_Per_Year'] = (df['Mileage_km'] / df['Age']).replace([np.inf, -np.inf], 0).fillna(0)
df['Mileage_Per_Year'] = df['Mileage_Per_Year'].astype('int64')
df['Is_Automatic'] = (df['Gearbox'].str.contains('Automatic|Semi-automatic', case=False, na=False)).astype(int)
df['Premium_Features'] = df['Full_Service_History'].astype(int) + df['Non_Smoker_Vehicle'].astype(int)

In [ ]:
numeric_df = df.select_dtypes(include='number')

df_corr = numeric_df.corr().round(2)

plt.figure(figsize=(16,6))
sns.heatmap(df_corr, annot=True, cmap='coolwarm', fmt='0.2f', linewidths = 0.5 , 
            annot_kws={'size': 16})
plt.show()

Now let's make logarithmic all the values that are right skewed.

In [ ]:
numerical_cols = [cname for cname in df.columns if df[cname].dtypes == 'int64' and cname != 'Price']

skew_df = pd.DataFrame(numerical_cols, columns =['Feature'])

skew_df['Skew'] = skew_df['Feature'].apply(lambda feature: scipy.stats.skew(df[feature]))

skew_df['Absolute Skew'] = skew_df['Skew'].apply(abs)

skew_df['Skewed'] = skew_df['Absolute Skew'].apply(lambda x: True if x >= 0.6 else False)

In [ ]:
skew_df

In [ ]:
for column in skew_df.query('Skewed == True')['Feature'].values:
    df[column] = np.log1p(df[column])

df['Price'] = np.log1p(df['Price'])

Not encoding model because i would have more than 1000 columns and risk that everything crashes

In [ ]:
df['Full_Service_History'] = df['Full_Service_History'].astype(int)
df['Non_Smoker_Vehicle'] = df['Non_Smoker_Vehicle'].astype(int)

categorical = ['Make', 'Body', 'Country', 'Condition', 
               'Fuel_Type', 'Drivetrain', 'Upholstery','Color']

df = df.drop(columns=("Gearbox"))

df_encoded = pd.get_dummies(df, columns=categorical, drop_first=True,dtype = int)

In [ ]:
df_encoded = df_encoded.drop(columns =("Model"))


## 3.2 Linear Regression 

Now that we have our encoded dataset, we can start with using linear regression and see what results we get, and later use different models to see what performs better.


In [ ]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

Splitting test and train data

In [ ]:
df_encoded.head(10)

In [ ]:
X = df_encoded.drop('Price', axis=1)
y = df_encoded['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
reg = LinearRegression()
reg.fit(X_train, y_train)

In [ ]:
pred = reg.predict(X_test)

sns.scatterplot(x = pred, y = y_test)
plt.title("Linear Regression")
plt.xlabel("Predictions")
plt.ylabel("Evaluation of our model")
plt.show()

In [ ]:
# Metrics in LOG scale (what the model actually optimizes)
scores_log = {
    "R2": r2_score(y_test, pred),
    "MAE": mean_absolute_error(y_test, pred),
    "RMSE": np.sqrt(mean_squared_error(y_test, pred))
}
print("Metrics in LOG scale:")
print(pd.DataFrame(scores_log, index=["Test"]))

# Metrics in ORIGINAL scale (what users care about)
y_test_original = np.exp(y_test)
pred_original = np.exp(pred)

scores_original = {
    "R2": r2_score(y_test_original, pred_original),
    "MAE": mean_absolute_error(y_test_original, pred_original),
    "RMSE": np.sqrt(mean_squared_error(y_test_original, pred_original))
}
print("\nMetrics in ORIGINAL scale (€):")
print(pd.DataFrame(scores_original, index=["Test"]))

As we can see we have some bad results, so let's see how many outliers are in our dataset and 
remove the ones where we have only few data about.

In [ ]:
price_original = np.exp(df_encoded['Price'])

print(f"Cars < 1000€: {(price_original < 1000).sum()}")
print(f"Cars < 3000€: {(price_original < 3000).sum()}")
print(f"Cars > 200k€: {(price_original > 200000).sum()}")
print(f"Cars > 500k€: {(price_original > 500000).sum()}")
print(f"Cars > 1M€: {(price_original > 1000000).sum()}")

let's remove the cars with price >500k and price < 3k and try again

In [ ]:
df_encoded['Price'] = np.exp(df_encoded['Price'])
df_encoded = df_encoded[(df_encoded['Price'] > 3000) & (df_encoded['Price'] < 500000)]

df_encoded['Price'] = np.log1p(df_encoded['Price'])

X = df_encoded.drop('Price', axis=1)
y = df_encoded['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
reg = LinearRegression()
reg.fit(X_train, y_train)
pred = reg.predict(X_test)


scores_log = {
    "R2": r2_score(y_test, pred),
    "MAE": mean_absolute_error(y_test, pred),
    "RMSE": np.sqrt(mean_squared_error(y_test, pred))
}
print("Metrics in LOG scale:")
print(pd.DataFrame(scores_log, index=["Test"]))


y_test_original = np.exp(y_test)
pred_original = np.exp(pred)

scores_original = {
    "R2": r2_score(y_test_original, pred_original),
    "MAE": mean_absolute_error(y_test_original, pred_original),
    "RMSE": np.sqrt(mean_squared_error(y_test_original, pred_original))
}
print("\nMetrics in ORIGINAL scale (€):")
print(pd.DataFrame(scores_original, index=["Test"]))

linear_r2 = scores_original['R2']
linear_mae = scores_original['MAE']
linear_rmse = scores_original['RMSE']

We get better results, but we can do more.

## 3.3 Forest Regressor

Since cars prices aren't dictated by linear correlations in most cases, we can use a random forest regressor which uses an ensemble of decision trees that can capture non-linear relationships and complex interactions between features.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor()

forest.fit(X_train, y_train)

In [ ]:
y_pred_log = forest.predict(X_test)


y_test_real = np.exp(y_test)
y_pred_real = np.exp(y_pred_log)

forest_r2 = r2_score(y_test_real, y_pred_real)
forest_mae = mean_absolute_error(y_test_real, y_pred_real)
forest_rmse = np.sqrt(mean_squared_error(y_test_real, y_pred_real))

print(f"--- Random Forest Performance ---")
print(f"R2 Score: {forest_r2:.4f}")
print(f"MAE (Euro): €{forest_mae:,.2f}")
print(f"RMSE (Euro): €{forest_rmse:,.2f}")

Let's look at which are considered the most important values to predict price

In [ ]:
feat_importances = pd.Series(forest.feature_importances_, index=forest.feature_names_in_)
top_30 = feat_importances.nlargest(30)

plt.figure(figsize=(10, 8))
sns.barplot(x=top_30.values, y=top_30.index, palette='viridis')
plt.show()

From this barplot we can see how power and age are the most important features, together with km, as we expected from our data exploration.

## 3.4 CatBoost

For our last regressor, let's use CatBoost, it uses decision trees but the way it differs from a random forest is that for every tree it corrects the errors made by the previous trees.
It also handles categorical values natively, so we can now include cars' model in our regressor.

In [ ]:
from catboost import CatBoostRegressor

categorical = ['Make', 'Model', 'Body', 'Country', 'Condition', 'Fuel_Type', 'Drivetrain',
              'Color', 'Upholstery']

# Using 3k-500k range
df['Price'] = np.exp(df['Price'])
df = df[(df['Price'] > 3000) & (df['Price'] < 500000)]
df['Price'] = np.log1p(df['Price'])


X = df.drop('Price', axis=1)
y = df['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

cat = CatBoostRegressor(learning_rate=0.03, iterations=2000,depth=7,
                        cat_features=categorical,loss_function='RMSE',verbose=200,
                        random_seed=42)

cat.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50)

In [ ]:
y_pred = cat.predict(X_test)
y_pred_original = np.exp(y_pred)
y_test_original = np.exp(y_test)

cat_r2 = r2_score(y_test_original, y_pred_original)
cat_mae = mean_absolute_error(y_test_original, y_pred_original)
cat_rmse = np.sqrt(mean_squared_error(y_test_original, y_pred_original))

print(f"--- Catboost Performance ---")
print(f"R2 Score: {cat_r2:.4f}")
print(f"MAE (Euro): €{cat_mae:,.2f}")
print(f"RMSE (Euro): €{cat_rmse:,.2f}")

## 3.5 Final Results

In [ ]:
data = {
    'Linear Regression': {
        'R2 Score': linear_r2,
        'MAE (€)': linear_mae,
        'RMSE (€)': linear_rmse
    },
    'Random Forest': {
        'R2 Score': forest_r2,
        'MAE (€)': forest_mae,
        'RMSE (€)': forest_rmse
    },
    'CatBoost': {
        'R2 Score': cat_r2,
        'MAE (€)': cat_mae,
        'RMSE (€)': cat_rmse
    }
}

pd.DataFrame(data).T.style.format({
    'R2 Score': '{:.4f}',
    'MAE (€)': '€{:,.2f}',
    'RMSE (€)': '€{:,.2f}'
})